In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
import tensorflow as tf

# Define the categories
crops = ['Maize', 'Wheat', 'Rice', 'Soybeans']
farming_types = ['Conventional', 'Organic', 'Hydroponic', 'Vertical']
districts = ['Bugesera', 'Gatsibo', 'Kayonza', 'Kirehe', 'Ngoma', 'Nyagatare', 'Rwamagana', 'Kicukiro', 'Gasabo', 'Nyarugenge', 'Burera', 'Gakenke', 'Gicumbi', 'Musanze', 'Rulindo', 'Gisagara', 'Huye', 'Kamonyi','Muhanga','Nyamagabe', 'Nyanza', 'Nyaruguru', 'Ruhango', 'Karongi', 'Ngororero', 'Nyabihu', 'Nyamasheke', 'Rubavu', 'Rusizi', 'Rutsiro']
fertilizers = ['Urea', 'DAP', 'MOP', 'NPK']
soil_types = ['Loamy', 'Sandy', 'Silty', 'Clay']

# Generate random data
data = {
    'Crop': np.random.choice(crops, 1000),
    'Farming Type': np.random.choice(farming_types, 1000),
    'District': np.random.choice(districts, 1000),
    'Fertilizer': np.random.choice(fertilizers, 1000),
    'Soil Type': np.random.choice(soil_types, 1000),
    'Area': np.random.uniform(1, 100, 1000),
    'Yield': np.random.uniform(10, 100, 1000)  # This is a placeholder. You might want to adjust this based on the crop yield per area in your region.
}

# Create a DataFrame
df = pd.DataFrame(data)

# Separate features and target
features = df.drop('Yield', axis=1)
target = df['Yield']

# Split the data into training and test sets
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Define preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['Area']),
        ('cat', OneHotEncoder(), ['Crop', 'Farming Type', 'District', 'Fertilizer', 'Soil Type'])
    ])

# Preprocessing the features
features_train = preprocessor.fit_transform(features_train).toarray()
features_test = preprocessor.transform(features_test).toarray()

# Define the model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=[features_train.shape[1]]),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])

# Compile the model
model.compile(loss='mean_squared_error',
              optimizer=tf.keras.optimizers.RMSprop(0.001),
              metrics=['mean_absolute_error', 'mean_squared_error'])

# Train the model
model.fit(features_train, target_train, epochs=10)

# Make predictions
predictions = model.predict(features_test)

# Convert predictions to a DataFrame
predictions_df = pd.DataFrame(predictions, columns=['Predicted Yield'])

print(predictions_df.head())



In [ ]:
def predict_yield(new_data):
    # Ensure new_data is a DataFrame
    if not isinstance(new_data, pd.DataFrame):
        new_data = pd.DataFrame(new_data)

    # Preprocess the new data
    new_data_preprocessed = preprocessor.transform(new_data).toarray()

    # Make prediction
    new_prediction = model.predict(new_data_preprocessed)

    return new_prediction

# Define new data
new_data = {
    'Crop': ['Maize'],
    'Farming Type': ['Conventional'],
    'District': ['Bugesera'],
    'Fertilizer': ['Urea'],
    'Soil Type': ['Loamy'],
    'Area': [50]
}

# Convert new data to DataFrame
new_data_df = pd.DataFrame(new_data)

# Make prediction
new_yield = predict_yield(new_data_df)

print('Predicted Yield:', new_yield)
